In [1]:
import random
import pandas as pd
import numpy as np
import Covid19_model as CM
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import os.path
import math
import csv
from pyomo.environ import *
import lhsmdu
import time

# Set parameter

In [2]:
D = 1e2
n_site = 12
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.3e6

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne, num_sample = 0, 0, 0, 20000
done = False
index = [0]
start_time = 6

for t in range(T):
    pset = env.para_sampling(100)
    para = pset[index[0]]
    action_buffer = [] 
    reward_buffer = []
    
    if budget > 0 and t > start_time:     
        for j in range(num_sample):  
            b0 = budget
            s0 = score
            ac = [random.random(),random.random(),random.random(),random.random(),random.random()]
            ac = [ac * n_site]
            ac = np.reshape(np.array(ac), (n_site, 5)).T
            
            action_buffer.append(ac) 
            state_ex, reward, _, _, c = env.transition(state, ac, para)  
            s0 += reward
            b0 -= c
            for i in range(t, T):
                if b0 > 0:
                    act = np.ones((5, n_site)) 
                    
                else:
                    act = np.zeros((5, n_site))
                next_ex, reward_ex, _, _, c = env.transition(state_ex, act, para)  
                s0 += reward_ex
                b0 -= c
                state_ex = next_ex.copy()          
            reward_buffer.append(s0)
               
        max_reward = np.argmax(np.array(reward_buffer))
        action = action_buffer[max_reward]
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, pset, 100, 1)

    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 Time: 0 | S: 22585916.0, E: 84.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-171.96
 Time: 0 | S: 22585916.0, E: 84.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-171.96
 Time: 0 | new_e:39.7, cum_e:39.7, new_i:35.5, cum_i:35.5
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 22585849.9, E: 110.9, A: 0.0, I: 35.5, D: 0.0, R: 3.8, Budget: 2300000.00, reward:-415.76
 Time: 1 | S: 22585851.0, E: 108.0, A: 0.0, I: 35.5, D: 0.0, R: 0.0, Budget: 2300000.00, reward:-415.76
 Time: 1 | new_e:55.7, cum_e:95.4, new_i:49.0, cum_i:84.5
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 2 | S: 22585757.0, E: 149.7, A: 0.0, I: 84.4, D: 0.0, R: 8.8, Budget: 230

 Time: 13 | S: 22373106.7, E: 307.8, A: 1342.6, I: 2209.5, D: 4.6, R: 209028.8, Budget: 2136922.01, reward:-14354.21
 Time: 13 | S: 22373273.0, E: 300.0, A: 1342.6, I: 2209.5, D: 4.6, R: 209023.0, Budget: 2136922.01, reward:-14354.21
 Time: 13 | new_e:163.0, cum_e:3237.1, new_i:285.5, cum_i:3393.5
[[0.10782919 0.10782919 0.10782919 0.10782919 0.10782919 0.10782919
  0.10782919 0.10782919 0.10782919 0.10782919 0.10782919 0.10782919]
 [0.71546598 0.71546598 0.71546598 0.71546598 0.71546598 0.71546598
  0.71546598 0.71546598 0.71546598 0.71546598 0.71546598 0.71546598]
 [0.9748019  0.9748019  0.9748019  0.9748019  0.9748019  0.9748019
  0.9748019  0.9748019  0.9748019  0.9748019  0.9748019  0.9748019 ]
 [0.98467588 0.98467588 0.98467588 0.98467588 0.98467588 0.98467588
  0.98467588 0.98467588 0.98467588 0.98467588 0.98467588 0.98467588]
 [0.1621698  0.1621698  0.1621698  0.1621698  0.1621698  0.1621698
  0.1621698  0.1621698  0.1621698  0.1621698  0.1621698  0.1621698 ]]
 Time: 14 | S: 22

 Time: 22 | S: 22117211.1, E: 132.9, A: 1096.6, I: 2730.3, D: 13.4, R: 464815.6, Budget: 1919378.28, reward:-21822.45
 Time: 22 | S: 22117429.0, E: 133.0, A: 1096.6, I: 2730.3, D: 13.4, R: 464817.0, Budget: 1919378.28, reward:-21822.45
 Time: 22 | new_e:87.8, cum_e:4836.6, new_i:190.3, cum_i:5886.1
[[0.1689025  0.1689025  0.1689025  0.1689025  0.1689025  0.1689025
  0.1689025  0.1689025  0.1689025  0.1689025  0.1689025  0.1689025 ]
 [0.75424578 0.75424578 0.75424578 0.75424578 0.75424578 0.75424578
  0.75424578 0.75424578 0.75424578 0.75424578 0.75424578 0.75424578]
 [0.97855835 0.97855835 0.97855835 0.97855835 0.97855835 0.97855835
  0.97855835 0.97855835 0.97855835 0.97855835 0.97855835 0.97855835]
 [0.93862847 0.93862847 0.93862847 0.93862847 0.93862847 0.93862847
  0.93862847 0.93862847 0.93862847 0.93862847 0.93862847 0.93862847]
 [0.23045075 0.23045075 0.23045075 0.23045075 0.23045075 0.23045075
  0.23045075 0.23045075 0.23045075 0.23045075 0.23045075 0.23045075]]
 Time: 23 | S: 

 Time: 31 | S: 21923286.8, E: 186.2, A: 749.5, I: 2744.1, D: 25.9, R: 659007.4, Budget: 1756758.87, reward:-28938.43
 Time: 31 | S: 21923512.0, E: 191.0, A: 749.5, I: 2744.1, D: 25.9, R: 659008.0, Budget: 1756758.87, reward:-28938.43
 Time: 31 | new_e:294.1, cum_e:6297.4, new_i:296.3, cum_i:7915.7
[[0.00414964 0.00414964 0.00414964 0.00414964 0.00414964 0.00414964
  0.00414964 0.00414964 0.00414964 0.00414964 0.00414964 0.00414964]
 [0.54580485 0.54580485 0.54580485 0.54580485 0.54580485 0.54580485
  0.54580485 0.54580485 0.54580485 0.54580485 0.54580485 0.54580485]
 [0.62311622 0.62311622 0.62311622 0.62311622 0.62311622 0.62311622
  0.62311622 0.62311622 0.62311622 0.62311622 0.62311622 0.62311622]
 [0.68533548 0.68533548 0.68533548 0.68533548 0.68533548 0.68533548
  0.68533548 0.68533548 0.68533548 0.68533548 0.68533548 0.68533548]
 [0.00986912 0.00986912 0.00986912 0.00986912 0.00986912 0.00986912
  0.00986912 0.00986912 0.00986912 0.00986912 0.00986912 0.00986912]]
 Time: 32 | S: 

 Time: 40 | S: 21772536.6, E: 283.5, A: 878.0, I: 2717.9, D: 36.2, R: 809547.8, Budget: 1596076.49, reward:-36783.52
 Time: 40 | S: 21772748.0, E: 288.0, A: 878.0, I: 2717.9, D: 36.2, R: 809564.0, Budget: 1596076.49, reward:-36783.52
 Time: 40 | new_e:189.6, cum_e:7955.9, new_i:250.9, cum_i:10124.9
[[0.02171805 0.02171805 0.02171805 0.02171805 0.02171805 0.02171805
  0.02171805 0.02171805 0.02171805 0.02171805 0.02171805 0.02171805]
 [0.25585694 0.25585694 0.25585694 0.25585694 0.25585694 0.25585694
  0.25585694 0.25585694 0.25585694 0.25585694 0.25585694 0.25585694]
 [0.94634278 0.94634278 0.94634278 0.94634278 0.94634278 0.94634278
  0.94634278 0.94634278 0.94634278 0.94634278 0.94634278 0.94634278]
 [0.84207047 0.84207047 0.84207047 0.84207047 0.84207047 0.84207047
  0.84207047 0.84207047 0.84207047 0.84207047 0.84207047 0.84207047]
 [0.04849238 0.04849238 0.04849238 0.04849238 0.04849238 0.04849238
  0.04849238 0.04849238 0.04849238 0.04849238 0.04849238 0.04849238]]
 Time: 41 | S:

 Time: 49 | S: 21585359.3, E: 90.4, A: 752.8, I: 2389.3, D: 46.6, R: 997361.6, Budget: 1381948.02, reward:-41705.27
 Time: 49 | S: 21585516.0, E: 91.0, A: 752.8, I: 2389.3, D: 46.6, R: 997418.0, Budget: 1381948.02, reward:-41705.27
 Time: 49 | new_e:60.4, cum_e:8973.5, new_i:130.6, cum_i:11797.8
[[0.2415473  0.2415473  0.2415473  0.2415473  0.2415473  0.2415473
  0.2415473  0.2415473  0.2415473  0.2415473  0.2415473  0.2415473 ]
 [0.93604328 0.93604328 0.93604328 0.93604328 0.93604328 0.93604328
  0.93604328 0.93604328 0.93604328 0.93604328 0.93604328 0.93604328]
 [0.95476593 0.95476593 0.95476593 0.95476593 0.95476593 0.95476593
  0.95476593 0.95476593 0.95476593 0.95476593 0.95476593 0.95476593]
 [0.9804557  0.9804557  0.9804557  0.9804557  0.9804557  0.9804557
  0.9804557  0.9804557  0.9804557  0.9804557  0.9804557  0.9804557 ]
 [0.13844193 0.13844193 0.13844193 0.13844193 0.13844193 0.13844193
  0.13844193 0.13844193 0.13844193 0.13844193 0.13844193 0.13844193]]
 Time: 50 | S: 2157

 Time: 58 | S: 21422077.5, E: 111.6, A: 397.6, I: 1765.0, D: 54.2, R: 1161594.0, Budget: 1188156.31, reward:-44970.03
 Time: 58 | S: 21422247.0, E: 117.0, A: 397.6, I: 1765.0, D: 54.2, R: 1161642.0, Budget: 1188156.31, reward:-44970.03
 Time: 58 | new_e:60.2, cum_e:9617.9, new_i:94.7, cum_i:12878.8
[[0.05332451 0.05332451 0.05332451 0.05332451 0.05332451 0.05332451
  0.05332451 0.05332451 0.05332451 0.05332451 0.05332451 0.05332451]
 [0.73833843 0.73833843 0.73833843 0.73833843 0.73833843 0.73833843
  0.73833843 0.73833843 0.73833843 0.73833843 0.73833843 0.73833843]
 [0.94297369 0.94297369 0.94297369 0.94297369 0.94297369 0.94297369
  0.94297369 0.94297369 0.94297369 0.94297369 0.94297369 0.94297369]
 [0.99977022 0.99977022 0.99977022 0.99977022 0.99977022 0.99977022
  0.99977022 0.99977022 0.99977022 0.99977022 0.99977022 0.99977022]
 [0.97566226 0.97566226 0.97566226 0.97566226 0.97566226 0.97566226
  0.97566226 0.97566226 0.97566226 0.97566226 0.97566226 0.97566226]]
 Time: 59 | S:

 Time: 67 | S: 21342104.8, E: 66.9, A: 260.0, I: 1175.4, D: 59.6, R: 1242333.4, Budget: 1032762.84, reward:-46878.28
 Time: 67 | S: 21342263.0, E: 70.0, A: 260.0, I: 1175.4, D: 59.6, R: 1242377.0, Budget: 1032762.84, reward:-46878.28
 Time: 67 | new_e:32.4, cum_e:9990.5, new_i:56.6, cum_i:13516.8
[[0.28218307 0.28218307 0.28218307 0.28218307 0.28218307 0.28218307
  0.28218307 0.28218307 0.28218307 0.28218307 0.28218307 0.28218307]
 [0.95170139 0.95170139 0.95170139 0.95170139 0.95170139 0.95170139
  0.95170139 0.95170139 0.95170139 0.95170139 0.95170139 0.95170139]
 [0.98493205 0.98493205 0.98493205 0.98493205 0.98493205 0.98493205
  0.98493205 0.98493205 0.98493205 0.98493205 0.98493205 0.98493205]
 [0.99189915 0.99189915 0.99189915 0.99189915 0.99189915 0.99189915
  0.99189915 0.99189915 0.99189915 0.99189915 0.99189915 0.99189915]
 [0.78523224 0.78523224 0.78523224 0.78523224 0.78523224 0.78523224
  0.78523224 0.78523224 0.78523224 0.78523224 0.78523224 0.78523224]]
 Time: 68 | S: 2

 Time: 76 | S: 21241071.4, E: 7.2, A: 127.0, I: 635.8, D: 62.2, R: 1344096.4, Budget: 810127.04, reward:-47503.27
 Time: 76 | S: 21241220.0, E: 12.0, A: 127.0, I: 635.8, D: 62.2, R: 1344142.0, Budget: 810127.04, reward:-47503.27
 Time: 76 | new_e:3.4, cum_e:10099.0, new_i:16.6, cum_i:13800.4
[[0.01001855 0.01001855 0.01001855 0.01001855 0.01001855 0.01001855
  0.01001855 0.01001855 0.01001855 0.01001855 0.01001855 0.01001855]
 [0.87340482 0.87340482 0.87340482 0.87340482 0.87340482 0.87340482
  0.87340482 0.87340482 0.87340482 0.87340482 0.87340482 0.87340482]
 [0.99944532 0.99944532 0.99944532 0.99944532 0.99944532 0.99944532
  0.99944532 0.99944532 0.99944532 0.99944532 0.99944532 0.99944532]
 [0.99775617 0.99775617 0.99775617 0.99775617 0.99775617 0.99775617
  0.99775617 0.99775617 0.99775617 0.99775617 0.99775617 0.99775617]
 [0.90965624 0.90965624 0.90965624 0.90965624 0.90965624 0.90965624
  0.90965624 0.90965624 0.90965624 0.90965624 0.90965624 0.90965624]]
 Time: 77 | S: 212478

 Time: 85 | S: 21022942.1, E: 5.1, A: 35.4, I: 270.0, D: 63.1, R: 1562684.3, Budget: 568341.40, reward:-47627.98
 Time: 85 | S: 21023082.0, E: 0.0, A: 35.4, I: 270.0, D: 63.1, R: 1562734.0, Budget: 568341.40, reward:-47627.98
 Time: 85 | new_e:2.6, cum_e:10117.2, new_i:6.1, cum_i:13881.6
[[0.78570785 0.78570785 0.78570785 0.78570785 0.78570785 0.78570785
  0.78570785 0.78570785 0.78570785 0.78570785 0.78570785 0.78570785]
 [0.953137   0.953137   0.953137   0.953137   0.953137   0.953137
  0.953137   0.953137   0.953137   0.953137   0.953137   0.953137  ]
 [0.99866364 0.99866364 0.99866364 0.99866364 0.99866364 0.99866364
  0.99866364 0.99866364 0.99866364 0.99866364 0.99866364 0.99866364]
 [0.98696733 0.98696733 0.98696733 0.98696733 0.98696733 0.98696733
  0.98696733 0.98696733 0.98696733 0.98696733 0.98696733 0.98696733]
 [0.05211866 0.05211866 0.05211866 0.05211866 0.05211866 0.05211866
  0.05211866 0.05211866 0.05211866 0.05211866 0.05211866 0.05211866]]
 Time: 86 | S: 20976559.8, 

In [6]:
(time.time() - tic) / ( T - start_time + 1)

77.19120998663061